In [1]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [2]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [3]:
Discrete(3)

Discrete(3)

In [4]:
Box(0,1,shape=(3,3)).sample()

array([[0.9359457 , 0.11586507, 0.8809493 ],
       [0.80558896, 0.21947074, 0.81902885],
       [0.5568509 , 0.65611017, 0.0932064 ]], dtype=float32)

In [5]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[137, 131,  60],
       [119, 166, 200],
       [111, 159,  60]])

In [6]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(1, array([78.70752], dtype=float32))

In [7]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([74.40019], dtype=float32))])

In [8]:
MultiBinary(4).sample()

array([1, 1, 0, 0], dtype=int8)

In [9]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 1], dtype=int64)

# 3. Building an Environment

In [10]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [11]:
env=ShowerEnv()

c:\Users\hhara\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [12]:
env.observation_space.sample()

array([97.959236], dtype=float32)

In [13]:
env.reset()

array([39.])

In [14]:
from stable_baselines3.common.env_checker import check_env

In [30]:
check_env(env, warn=True)

AssertionError: Your environment must inherit from the gymnasium.Env class cf. https://gymnasium.farama.org/api/env/

# 4. Test Environment

In [20]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-26
Episode:2 Score:-60
Episode:3 Score:-8
Episode:4 Score:-60
Episode:5 Score:0


In [21]:
env.close()

# 5. Train Model

In [22]:
log_path = os.path.join('Training', 'Logs')

In [23]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\hhara\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [26]:
model.learn(total_timesteps=4000)

Logging to Training\Logs\PPO_3


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 57.6     |
| time/              |          |
|    fps             | 2259     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 54.5        |
| time/                   |             |
|    fps                  | 1488        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.018234883 |
|    clip_fraction        | 0.15        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.604      |
|    explained_variance   | -4.65e-06   |
|    learning_rate        | 0.

# 6. Save Model

In [27]:
model.save('PPO')

In [28]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

c:\Users\hhara\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
c:\Users\hhara\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(59.2, 0.9797958971132712)